## ProcessDataForSubmittingTrainV7XGBでの判明事項
訓練データの20%の1293件のデータで検証した結果
-　平均値を超えている件数が436件あり、平均以上に引っ張られている形
- 800%, 400%外れているケースあり。これは同じ現場でも他の号棟と異なるために安くなっているケースがあった
- また、他の号棟に比べて面積(tc_mseki)が広いが、価格に反映できていないケースがある

⇒学習データに入っていないために、予測に反映できていないように見える。

　⇒学習と検証データに分けるパターンを複数作成し、平均もしくは中間の値から価格を決めるのはどうか

## はずれ値に関して調査する
1. 教師データから外れ値（⇒極端に安い物件）をピックアップする
1. 教師データに対して、その教師データで学習したモデルで予測し、その外れ値にどのような予測ができているか確認する

### みつかった外れ値物件
- pj_no = 2019   280万と2800万
- pj_no = 2192   300万と3000万
- pj_no = 758     350万と2800万
- pj_no = 2504   360万と3700万
- pj_no = 2412   550万と2500万
- pj_no = 2036  1000万と1700万

## わかったこと
- pj_no 2019 : niwasakiが同じpj_noのものに比べて半分 他：4.2  安いもの:2.7
- pj_no 2192 : setsudo_hiが他の物件と違う　　他：東　安い：南
- pj_no 758 : 他の物件より悪条件が多い　tc_msekiが極端にちいさい maguchi, niwasakiが極端にちいさい
- pj_no 2504 : 差異無し　判定無理
- pj_no 2412 : 安い物件もちゃんと判定できている9%の誤差。単に学習データに含まれているからかもしれない。

## やってみたいこと
- pj_no 758より ... tc_msekiとtc_mseki_avgとの比のカラムを追加する
- pj_no 758より ... mabuchiが狭いもの物件に対して、間口狭のカテゴリを１にする
- pj_no 1535より ... 当該pj_noで最も狭い物件にフラグを立てる
- pj_no 1047より ... 最大の面積を持つ2502がおかしい。tc_msekiの反映に失敗している様子。やはり単価で予測すべきか
- pj_no 408より ... levelplanをdropしているのがまずいと思われる。次はこれをやるべき。補完が難しいかもしれない。
- pj_no 1457より ... levelplanが土地売りのものを反映できていない。やはりdropしているのがまずい

### 2 ローカル向け処理
- X_train, X_evalは、idとpj_no列をdropしてから、訓練、評価に使う
- Y_train, Y_evalは、tanka_pr列を利用する
#### 教師データ：訓練用

In [1]:
# 共通処理
# x_train. y_train, x_eval, y_evalを作成する
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def mean_absolute_percentage_error( y_train: np.array, y_pred: np.array):
    diff = 0
    n = len(y_train)
    for i in range(n):
        diff += abs(y_train[i]-y_pred[i])/y_train[i]
    score = 100*diff / n
    
    return score


train_x = pd.read_csv("data/processed_train_goto_x_v7.csv")
train_y = pd.read_csv("data/processed_train_goto_y_v7.csv")
X_train, X_eval, Y_train, Y_eval = train_test_split( train_x, train_y, train_size=0.8, random_state = 19711022)

X_train.to_csv("data/X_train.csv", index=False)
X_eval.to_csv("data/X_eval.csv", index=False)
Y_train.to_csv("data/Y_train.csv", index=False)
Y_eval.to_csv("data/Y_eval.csv", index=False)

train_x = pd.read_csv('data/X_train.csv').drop(['id','pj_no'],axis=1)
train_y = pd.read_csv('data/Y_train.csv').drop(['id'],axis=1)

import xgboost as xgb
from xgboost import XGBRegressor
import time

params = {
    'n_estimators':700,
    'max_depth':6,
    'min_child_weight':9,
    'gamma':0,
    'subsample':1.0,
    'colsample_bytree':0.6,
    'learning_rate':0.1
}

print(f"start learning...")
xgboost_opt = XGBRegressor(**params, seed=42, n_jobs=-1)
start = time.perf_counter()
xgboost_opt.fit(train_x, train_y)
end = time.perf_counter()
print(end-start)

print(f"start estimating...")
eval_x = pd.read_csv('data/processed_train_goto_x_v7.csv').drop(['id','pj_no'],axis=1)
ans_y = pd.read_csv('data/processed_train_goto_y_v7.csv').drop(['id'],axis=1)
pred_y = xgboost_opt.predict(eval_x)
print( mean_absolute_percentage_error(ans_y.values,pred_y))

out = pd.read_csv('data/processed_train_goto_y_v7.csv')
out['pred_keiyaku_pr'] = pd.Series(pred_y).astype(np.int64)
out['error']=abs((out['keiyaku_pr']-out['pred_keiyaku_pr'])/out['keiyaku_pr'])*100
output = pd.merge(out, pd.read_csv("data/processed_train_goto_x_v7.csv"),on='id')
output.to_csv("data/train_data_error.csv")

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


start learning...
20.882111721
start estimating...
[ 3.65103007]


In [14]:
print(f"start estimating...")
eval_x = pd.read_csv('data/X_eval.csv').drop(['id','pj_no'],axis=1)
ans_y = pd.read_csv('data/Y_eval.csv').drop(['id'],axis=1)
pred_y = xgboost_opt.predict(eval_x)
print( mean_absolute_percentage_error(ans_y.values,pred_y))

out = pd.read_csv('data/Y_eval.csv')
out['pred_keiyaku_pr'] = pd.Series(pred_y).astype(np.int64)
out['error']=abs((out['keiyaku_pr']-out['pred_keiyaku_pr'])/out['keiyaku_pr'])*100
output = pd.merge(out, pd.read_csv("data/X_eval.csv"),on='id')
output.to_csv("data/eval_data_error.csv")

start estimating...
[ 8.81650293]


In [51]:
eval_x = pd.read_csv('data/X_eval.csv')

In [52]:
eval_x['pred_keiyaku_pr']=pd.Series(pred_y)

In [53]:
df =pd.merge(eval_x, pd.read_csv('data/Y_eval.csv'), on='id')

In [57]:
df['diff']=(abs(df['pred_keiyaku_pr']-df['keiyaku_pr'])/df['keiyaku_pr'])*100

In [58]:
df.to_csv('data/difference.csv')

### n_estimatorsが700のケースでsubmitしてみることにする(7/7)

In [27]:
test_x = pd.read_csv("data/processed_test_goto_x.csv")
test_pred = xgboost_opt.predict(test_x.drop(['id','pj_no'],axis=1))
submit = pd.DataFrame(test_x[['id']])
submit['keiyaku_pr']=pd.Series(test_pred).astype(np.int64)
submit.to_csv('data/submit4.tsv',sep='\t',header=None, index=False)

### ここからSageMaker用のデータを作る処理

In [ ]:
train_x = pd.read_csv('data/X_train.csv')
train_y = pd.read_csv('data/Y_train.csv')

In [ ]:
train_input = pd.concat([train_y.drop(['id','keiyaku_pr','tc_mseki'],axis=1),train_x.drop(['id','pj_no'],axis=1)],axis=1)
train_input.to_csv('data/sagemaker_input.csv', header=None, index=False)
eval_x = pd.read_csv('data/X_eval.csv')
eval_x.drop(['id','pj_no'],axis=1).to_csv('data/sagemaker_eval_input.csv',header=None, index=False)


### SageMakerの出力から精度を計算する

In [ ]:
pred2_y = pd.read_csv('data/sagemaker_eval_input.csv.out', header=None)
ans_y = pd.read_csv('data/Y_eval.csv').drop(['id','keiyaku_pr','tc_mseki'],axis=1)

In [ ]:
print( mean_absolute_percentage_error(ans_y.values,pred2_y.values))

### SageMaker用予測データを作成する

In [ ]:
test_x = pd.read_csv("data/processed_test_goto_x.csv")

In [ ]:
test_input = test_x.drop(['id','pj_no'],axis=1)
test_input.to_csv('data/sagemaker_test_input.csv', header=None, index=False)

### SageMaker出力からsubmit用データを作る

In [ ]:
tanka = pd.read_csv("data/sagemaker_test_input.csv.out", header=None )

In [ ]:
test_x = pd.read_csv("data/processed_test_goto_x.csv")

In [ ]:
submit = pd.DataFrame(test_x[['id', 'tc_mseki']])

In [ ]:
submit['tanka_pr']=tanka

In [ ]:
submit['price']=(submit['tc_mseki']*submit['tanka_pr']).astype(np.int64)

In [ ]:
submit.loc[:,['id','price']].to_csv('data/submit3.tsv',sep='\t',header=None, index=False)

In [ ]:
submit.head()